# OpenAI ChatCompletion API 를 비동기/병렬 요청 생성

병렬 요청/응답 취합
- OpenAI Cookbook 의 `api_request_parallel_processor.py` 를 이용
  - 요청 파일에 대해 비동기/병렬 요청
  - 응답 결과를 파일에 저장
  
# 요청 파일 준비

입력: 원본 엑셀 (또는 CSV 등) 파일

출력: OpenAI ChatCompletion API 요청을 위한 JSONL 파일


In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import json
import re

## API KEY는 `OPENAI_API_KEY` 환경변수를 통해 설정

In [2]:
import openai
import os
with open("../specs/.openai_api_key", "r") as f:
    ss = f.readline().strip()

os.environ["OPENAI_API_KEY"] = ss

## 설정

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
model_list = client.models.list()

이용 가능한 모델 조회

In [5]:
for model in model_list.data:
    print(f"Model ID: {model.id}, Created: {model.created}, Owned by: {model.owned_by}")

Model ID: gpt-4-1106-preview, Created: 1698957206, Owned by: system
Model ID: chatgpt-4o-latest, Created: 1723515131, Owned by: system
Model ID: tts-1-hd-1106, Created: 1699053533, Owned by: system
Model ID: tts-1-hd, Created: 1699046015, Owned by: system
Model ID: dall-e-2, Created: 1698798177, Owned by: system
Model ID: text-embedding-3-large, Created: 1705953180, Owned by: system
Model ID: gpt-4-0125-preview, Created: 1706037612, Owned by: system
Model ID: gpt-3.5-turbo-0125, Created: 1706048358, Owned by: system
Model ID: gpt-4o-mini, Created: 1721172741, Owned by: system
Model ID: gpt-4o-mini-2024-07-18, Created: 1721172717, Owned by: system
Model ID: gpt-4-turbo-preview, Created: 1706037777, Owned by: system
Model ID: gpt-3.5-turbo, Created: 1677610602, Owned by: openai
Model ID: tts-1-1106, Created: 1699053241, Owned by: system
Model ID: whisper-1, Created: 1677532384, Owned by: openai-internal
Model ID: gpt-4-turbo, Created: 1712361441, Owned by: system
Model ID: tts-1, Created

### 설정

모델 지정

원본 파일

요청 JSONL 파일

In [6]:
engine_id = 'gpt-4o-mini'
raw_file_path = r'../local_data/(2024-04-29)구글 디스커버_데이터_수집.xlsx'
api_request_file_path = r'../local_data/api_requests_for_annotated_dataset.jsonl'

In [7]:
all_sheets = pd.read_excel(raw_file_path, sheet_name=None)
new_cols = ['url-title', 'source', 'issue-dt', 'title', 'url', 'precision', 'freshness',
       'satisfaction', 'trial-index', 'dt']

In [8]:
all_sheets['김미령']

,URL + 제목,언론사,발행시간,제목,URL,정확도 (10),신선함 (10),만족도 (10),차수,일자
0,중국에서 잘 지내? 푸바오 최근 근황 #shorts / KBS https://www...,KBS News,10시간전,중국에서 잘 지내? 푸바오 최근 근황 #shorts / KBS,https://www.youtube.com/watch?v=3jOWZgTSQ1Q,8,8,10.0,0,412
1,이 2가지로 남들과 격차가 벌어집니다. 제가 만나본 부자들은 모두 이 방법을 그대로...,월급쟁이부자들TV,2024.4.12,이 2가지로 남들과 격차가 벌어집니다. 제가 만나본 부자들은 모두 이 방법을 그대로...,https://www.youtube.com/watch?v=DLKnlsOd9bk,8,7,7.0,0,412
2,양배추와 두부를 이렇게 드세요! 속편하고 든든한 양배추 요리 2가지👍💯 https:...,수리키친Suri,5일전,양배추와 두부를 이렇게 드세요! 속편하고 든든한 양배추 요리 2가지👍💯 ht,https://www.youtube.com/watch?v=Y2qZDy9qwrU,10,8,8.0,0,412
3,우리 푸바오는요...! | #푸바오 #강철원 #인터뷰 - YouTube https:...,에버랜드,1일전,우리 푸바오는요...! | #푸바오 #강철원 #인터뷰 - YouTube,https://www.youtube.com/watch?v=HmVaFdfU4qE,7,8,8.0,0,412
4,"시스루에 다 비쳐…'워터밤 여신' 권은비, 87만원으로 완성한 퇴폐美 https:/...",10텐아시아,2024.4.8,"시스루에 다 비쳐…'워터밤 여신' 권은비, 87만원으로 완성한 퇴폐美",https://tenasia.hankyung.com/article/202404086...,5,5,5.0,0,412
...,...,...,...,...,...,...,...,...,...,...
552,"회장님의 귀환…김승연, 세 아들 사업장 돌며 '건재 과시'한 이유 https://n...",머니투데이,7시간,"회장님의 귀환…김승연, 세 아들 사업장 돌며 '건재 과시'한 이유",https://news.mt.co.kr/mtview.php?no=2024042614...,8,10,10.0,7,427
553,정연 “나 자신을 이기는 게 너무 재밌어요” https://www.gqkorea.c...,지큐 코리아(GQ Korea),3일,정연 “나 자신을 이기는 게 너무 재밌어요”,https://www.gqkorea.co.kr/2024/04/24/%EC%A0%95...,2,2,2.0,7,427
554,월드컵팀까지 ‘투잡’ 뛴 황선홍... 준비 소홀이 부른 ‘도하 참극’ https:/...,조선일보,1일,월드컵팀까지 ‘투잡’ 뛴 황선홍... 준비 소홀이 부른 ‘도하 참극’,https://www.chosun.com/sports/football/2024/04...,3,5,5.0,7,427
555,방시혁-민희진 갈등 와중에…뉴진스 새 싱글 앨범 공개 https://www.hani...,한겨레,23시간,방시혁-민희진 갈등 와중에…뉴진스 새 싱글 앨범 공개,https://www.hani.co.kr/arti/culture/culture_ge...,2,2,2.0,7,427


In [20]:
def make_aggregated_df(all_sheets, key):
    """
    2024-08월 부터 gpt-4o, gpt-4o-mini 의 출력이 16K 출력으로 (기존 4K)에서 증가 되었으므로
    한번 요청에 하루의 모든 콘텐츠에 대한 문의를 담을 수 있다.
    """
    
    df = all_sheets[key]
    df.columns = new_cols
    
    def add_serial_numbers_and_chunk(group, start_offset=0):
        numbered_titles = [f"{start_offset+j+1}. {title}" for j, title in enumerate(group)]
        return '\n'.join(numbered_titles)

    # 새로운 DataFrame을 만들기 위한 리스트
    result = []
    
    # 그룹화하여 처리
    start_offset=0
    grouped = df.groupby('trial-index')['title']
    for trial_index, group in grouped:
        chunk = add_serial_numbers_and_chunk(group.tolist(), start_offset)
        result.append((trial_index, chunk))
        start_offset += len(group)
    
    # 결과 DataFrame 생성
    result_df = pd.DataFrame(result, columns=['trial-index', 'titles'])
    
    return result_df

# Prompt Template

시스템 프롬프트

In [31]:
system_prompt = """뉴스 기사에 대한 카테고리를 다음과 같이 정의 했어. 잘 기억해둬.

1. 정치
   1.1. 국내 정치
       1.1.1. 청와대/행정부
       1.1.2. 국회/입법부
       1.1.3. 지방자치
       1.1.4. 선거
   1.2. 국제 정치
       1.2.1. 외교/국제관계
       1.2.2. 국제기구 (UN, NATO, EU 등)
       1.2.3. 정상회담/국제회의
   1.3. 북한
       1.3.1. 남북관계
       1.3.2. 북한 정세
       1.3.3. 북핵 문제
   1.4. 군사/안보
       1.4.1. 국방 정책
       1.4.2. 무기/방위산업
       1.4.3. 사이버 안보
       1.4.4. 테러리즘

2. 경제
   2.1. 금융
       2.1.1. 글로벌 금융시장
   2.2. 증권/주식
   2.3. 부동산
   2.4. 산업/기업
       2.4.1. 글로벌 기업 동향
   2.5. 중소기업/창업
   2.6. 경제 정책
       2.6.1. 국제 경제 정책

3. 사회
   3.1. 사건사고
       3.1.1. 국제 사건사고
   3.2. 교육
   3.3. 노동
   3.4. 환경
       3.4.1. 글로벌 환경 이슈
   3.5. 인권/복지
       3.5.1. 국제 인권

4. 생활/문화
   4.1. 건강
   4.2. 요리/맛집
   4.3. 패션/뷰티
   4.4. 여행/레저
   4.5. 자동차
   4.6. 책/문학
   4.7. 종교
   4.8. 육아/교육
   4.9. 반려동물
   4.10. 취미/DIY
   4.11. 웨딩/결혼

5. IT/과학
   5.1. 모바일
   5.2. 인터넷/SNS
   5.3. 통신/뉴미디어
   5.4. IT 기기
   5.5. 과학 일반
   5.6. 우주/항공
   5.7. 인공지능
   5.8. 블록체인/가상화폐
   5.9. 바이오테크놀로지 

6. 세계
   6.1. 아시아/호주
   6.2. 미국/중남미 
   6.3. 유럽
   6.4. 중동/아프리카
   6.5. 국제분쟁/전쟁
   6.6. 세계 일반

7. 연예
   7.1. 영화
   7.2. 방송/드라마
   7.3. 음악
   7.4. 스타/연예인
   7.5. 애니메이션/웹툰
   7.6. 셀러브리티/인플루언서

8. 스포츠
   8.1. 국내 야구
   8.2. 해외 야구
   8.3. 국내 축구 
   8.4. 해외 축구
   8.5. 골프
   8.6. 농구
   8.7. 배구
   8.8. e스포츠
   8.9. 테니스
   8.10. 격투기 (복싱, UFC 등)
   8.11. 모터스포츠 (F1, 랠리 등)
   8.12. 익스트림 스포츠

9. 오피니언
   9.1. 사설
   9.2. 칼럼
   9.3. 기자수첩
   9.4. 만평

10. 날씨
    10.1. 일기예보
    10.2. 기상이슈
    10.3. 계절/시즌 정보

11. 지역
    11.1. 수도권
    11.2. 강원/충청
    11.3. 전라/경상
    11.4. 제주

12. 행사/이벤트 
    12.1. 축제/페어
    12.2. 전시/공연
    12.3. 세미나/포럼
    12.4. 공모전/이벤트

13. 인물/인터뷰
    13.1. 정치인
    13.2. 기업인
    13.3. 학자/전문가
    13.4. 활동가/사회운동가

14. HistoHistory/역사
    14.1. 한국사
    14.2. 세계사
    14.3. 근현대사
    14.4. 문화유산/유물

### 사용자에게 제공된 개인화 뉴스 컨텐츠들의 제목을 검토한 후 핵심 키워드와 뉴스 카테고리를 할당해줘. 
입력 양식은 다음과 같아.
- 하나의 행에 "일련번호. 뉴스 제목" 순으로 되어 있어.

다음은 요구사항이야.
1. 뉴스 제목에서 핵심 키워드를 추출하자. 여러가지면 더 좋다.
2. 뉴스 제목에 유추할 수 있는 세부 카테고리 오직 하나를 할당한다.

출력 양식은 다음과 같이 해줘.
입력에 적힌 "일련번호" : "번호를 포함한 세부 카테고리명" : "키워드1" : "키워드2" : "키워드3", ... 와 같은 포맷으로 라인마다 하니씩 출력해줘.

만약 입력에 대해 분석할게 없거나 분석을 못할 경우에는
입력에 적힌 일련번호 : None 으로 출력해서 전체적으로 누락되는게 없게 해줘.
"""

유저 프롬프트

In [32]:
user_prompt = """{0}
"""

# 비동기 요청 포맷

## 참고: metadata

`request_id` 
- 비동기/병렬 요청시 나중에 요청한 응답이 먼저 도착할 수도 있음.
- metadata.request_id 등을 추가해두면, 이를 포함하여 응답이 리턴됨.
- 취합시 이를 활용한 정렬 가능


In [33]:
def make_json_body(request_id, system_prompt, user_prompt):
    dic = { 
        'model':engine_id,
        'messages':[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        'temperature':0.0,
        'metadata':{'request_id':request_id},
    }
    return json.dumps(dic, ensure_ascii=False)

In [34]:
with open(api_request_file_path, 'w') as f:
    cnt = 0
    sheet_names = all_sheets.keys()
    for key in sheet_names:
        df = make_aggregated_df(all_sheets, key)
        
        for i, row in df.iterrows():
            jsonl = make_json_body(
                request_id=f"{key}:{row['trial-index']}:{i}",
                system_prompt=system_prompt,
                user_prompt=user_prompt.format(row.titles)
                )      
            f.write(jsonl + '\n')
            
        cnt += len(df)

# 요청 수행

예시

```bash
#!/bin/bash

if [[ -z "$OPENAI_API_KEY" ]]; then
  echo "Error: OPENAI_API_KEY is not set. Please set the API key and try again."
  exit 1
fi

python src/openai/api_request_parallel_processor.py \
  --requests_filepath local_data/api_requests_for_annotated_dataset.jsonl\
  --save_filepath local_data/api_responses_for_annotated_dataset.jsonl \
  --request_url https://api.openai.com/v1/chat/completions \
  --api_key $OPENAI_API_KEY \
  --max_requests_per_minute 500 \
  --max_tokens_per_minute 200000 \
  --max_attempts 5 \
  --logging_level 20
  ```